In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmbd-datasets/tmdb_5000_movies.csv
/kaggle/input/tmbd-datasets/tmdb_5000_credits.csv


In [2]:
movies=pd.read_csv("/kaggle/input/tmbd-datasets/tmdb_5000_movies.csv")
credit=pd.read_csv("/kaggle/input/tmbd-datasets/tmdb_5000_credits.csv")

In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
movies=movies.merge(credit, on='title')
movies.shape

(4809, 23)

In [5]:
print(movies['title'][0])

Avatar


In [6]:
movies=movies[['id', 'original_title', 'genres', 'keywords', 'cast', 'crew', 'overview']]

In [7]:
movies.dropna(inplace=True)
movies.isnull().sum()

id                0
original_title    0
genres            0
keywords          0
cast              0
crew              0
overview          0
dtype: int64

In [8]:
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
import ast
def convert(obj):
    l=[]
    #ast.literal_eval() convert a string into list
    for i in ast.literal_eval(obj) :
        l.append(i['name'])
    return l

In [10]:
movies['genres']=movies['genres'].apply(convert) # as movies['genres'] that's a string you have to write it lke that

In [11]:
movies['keywords']=movies['keywords'].apply(convert) # as movies['genres'] that's a string you have to write it lke that

In [12]:
def convert_cast(obj):
    l=[]
    cnt=0
    for i in ast.literal_eval(obj) :
        if cnt!=3:
            l.append(i['name'])
            cnt=cnt+1
        else:
            break
    return l

In [13]:
movies['cast']=movies['cast'].apply(convert_cast)

In [14]:
x=movies['crew'].apply(ast.literal_eval)

In [15]:
def detach_director(obj):
    l=[]
    cnt=0
    for i in ast.literal_eval(obj) :
        if i['job']=='Director':
            l.append(i['name'])
    return l

In [16]:
movies['crew']=movies['crew'].apply(detach_director)

In [17]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [18]:
movies.head()

,id,original_title,genres,keywords,cast,crew,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [19]:
vanish_space=lambda x:[i.replace(" ", "")for i in x]
movies['genres']=movies['genres'].apply(vanish_space)

In [20]:
movies['genres'][0]

['Action', 'Adventure', 'Fantasy', 'ScienceFiction']

In [21]:
movies['crew']=movies['crew'].apply(vanish_space)
movies['keywords']=movies['keywords'].apply(vanish_space)
movies['cast']=movies['cast'].apply(vanish_space)

In [22]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [23]:
new_df=movies[['id','original_title','tags']]

In [24]:
new_df.head()

,id,original_title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [25]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipykernel_17/487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [26]:
new_df['tags']=[s.lower() for s in new_df['tags']]

/tmp/ipykernel_17/2707754272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=[s.lower() for s in new_df['tags']]


In [27]:
import nltk
print(nltk.data.path)

['/root/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [28]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [29]:
new_df['tags']=new_df['tags'].apply(stem)

/tmp/ipykernel_17/3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
vectors=cv.fit_transform(new_df['tags']).toarray()

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
similarity= cosine_similarity(vectors)

In [32]:
def recommender(movie):
    movie_index=new_df[new_df['original_title']==movie].index[0]
    distance= similarity[movie_index]
    movie_list=sorted(list(enumerate(distance)), reverse=True, key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].original_title)

In [33]:
recommender('Avatar')

Aliens vs Predator: Requiem
Aliens in the Attic
Independence Day
Aliens
Battle: Los Angeles


In [34]:
import pickle
# Save the dictionary to a .pkl file
output_path = '/kaggle/working/movie_name.pkl'
pickle.dump(new_df.to_dict(), open(output_path, 'wb'))

print(f"File saved to {output_path}")


File saved to /kaggle/working/movie_name.pkl


In [35]:
output_path = '/kaggle/working/similarity.pkl'
pickle.dump(similarity,open(output_path, 'wb'))
print(f"File saved to {output_path}")

File saved to /kaggle/working/similarity.pkl
